In [1]:
import pandas as pd
import numpy as np
from scipy.io import loadmat
from scipy.sparse import csc
import os
import glob
import pickle

In [2]:
a = loadmat('network/Adj_mat.mat')
names = [arr[0][0] for arr in a["GeneSymbol_net"]]
Adj_mat = a["Adj_mat"]
D_mat_half_inv = csc.csc_matrix(np.diag((1./np.sqrt(np.array(Adj_mat.sum(axis=0)))).flatten()))
#D_mat_half_inv = np.diag(1./np.sqrt(np.sum(Adj_mat, axis = 0)))

D_V = csc.csc_matrix(np.eye(len(names)))
W_V = D_mat_half_inv@Adj_mat@D_mat_half_inv
L_V = D_V - W_V

In [3]:
output_save_dir = "output"
try:
    os.mkdir(output_save_dir)
except:
    print("Directory already created")

Directory already created


In [4]:
all_cancer_files = glob.glob("input_cancer_data/*.mat")
cancer_names = [file.split("\\")[1].split(".")[0].split("_")[0] for file in all_cancer_files]

In [5]:
for idx, file in enumerate(all_cancer_files):
    print("Run "+ file)
    d = loadmat(file)

    X_mut  = ((d["X_SNV_Net"]!=0)+0)
    X_exp = d["X_EXP_Net"][:, np.ravel(np.sum(X_mut.todense(), axis=0))>0]
    N_sample = X_mut.shape[0]

    PearsonCor = np.corrcoef(X_exp)
    sigma_bandwidth = 1.0
    W_U = np.exp(-((1.-PearsonCor)**2)/(2.*sigma_bandwidth**2))
    D_U = np.diag(np.sum(W_U,axis = 0))
    L_U = D_U - W_U

    K_num = 4
    lambda_LU = 1.0
    lambda_RU = 1.0
    lambda_LV = 1.0
    lambda_RV = 1.0

    # --- RS-CRNMF --- %
    #print('run RS-CRNMF ...')
    eps_t = 10**(-5)


    U_init = np.maximum(np.random.rand(N_sample,K_num),eps_t)
    U_prev = U_init@np.diag(np.sum(U_init,axis = 0)**(-1));
    V_init = np.maximum(np.random.rand(len(names),K_num),eps_t);
    V_prev = V_init@np.diag(np.sum(U_init, axis = 0));
    delta_U = 1.;
    delta_V = 1.;
    cnt = 0;
    while ((delta_U > 10**(-3)) or (delta_V > 10**(-3)) and (cnt <= 500)):
        cnt = cnt + 1;
        V_numer = X_mut.T@U_prev + lambda_LV*W_V@V_prev;
        V_denum = V_prev@(U_prev.T@U_prev) + lambda_LV*D_V@V_prev + lambda_RV*np.ones((len(names),len(names)))@V_prev;
        V_new = V_prev*V_numer/(V_denum + eps_t);
        #print("V_new ", V_new.shape)
        U_numer = X_mut@V_new + lambda_LU*W_U@U_prev;
        U_denum = U_prev@(V_new.T@V_new) + lambda_LU*D_U@U_prev + lambda_RU*U_prev;
        U_new = U_prev*U_numer/(U_denum + eps_t);
        #print("U_new "U_new.shape)
        NormFactor = np.sum(U_new,axis = 0);
        #print(NormFactor.shape)
        U_new = U_new@np.diag(NormFactor**(-1));
        V_new = V_new@np.diag(NormFactor);

        delta_U = np.linalg.norm(U_prev - U_new,'fro')**2/(np.linalg.norm(U_prev,'fro')**2);
        delta_V = np.linalg.norm(V_prev - V_new,'fro')**2/(np.linalg.norm(V_prev,'fro')**2);

        U_prev = U_new;
        V_prev = V_new;
    top_genes = list(pd.Series(names)[np.flip(np.argsort(np.sort(V_new, axis = 1)[:, -1]))[:200]])
    res = {"top_genes": top_genes, "U_new":U_new, "V_new":V_new}
    with open(output_save_dir+"/"+cancer_names[idx]+".pk", 'wb') as f:
        pickle.dump(res, f)
    

Run input_cancer_data\brca_tcga_pub.mat
Run input_cancer_data\coadread_tcga_pub.mat
Run input_cancer_data\gbm_tcga_pub.mat


In [11]:
output_matlab_genes = glob.glob("output/*.mat")
output_python_genes = glob.glob("output/*.pk")
compare_results = {}
for idx in range(len(output_python_genes)):
    name_of_cancer = output_python_genes[idx].split("\\")[1].split(".")[0]
    matlab_data = loadmat(output_matlab_genes[idx])
    with open(output_python_genes[idx], 'rb') as f:
        python_data = pickle.load(f)
    compare_results[name_of_cancer] = [matlab_data['Candidates_list'], python_data["top_genes"]]

In [17]:
pd.DataFrame.from_dict(compare_results)

,brca,coadread,gbm
0,"[[[TP53]], [[PIK3CA]], [[TTN]], [[MUC16]], [[G...","[[[APC]], [[TP53]], [[KRAS]], [[TTN]], [[SYNE1...","[[[TP53]], [[PTEN]], [[EGFR]], [[NF1]], [[RB1]..."
1,"[TP53, PIK3CA, TTN, MUC16, GATA3, CDH1, MAP3K1...","[APC, TP53, KRAS, TTN, SYNE1, LRP1B, MUC16, PI...","[TP53, PTEN, EGFR, NF1, RB1, PIK3R1, ERBB2, PI..."
